# Simulations for the MESS+ estimator

In [1]:
import yaml

from pathlib import Path
from pprint import pprint

from classifier.model import MultilabelBERTClassifier
from classifier.file_reader import read_files_from_folder



In [2]:
BENCHMARK_NAME = "boolq"
# this refers to whether we want to use a pre-trained classifier or learn the classifier online while benchmarking.
APPROACH = "pretrained"  # alt: online
# This only has an effect when APPROACH = pretrained. Make sure to adjust the minibatch size accordingly!
NUM_PRETRAINING_STEPS = 200

PROJECT_ROOT_PATH = Path("mess_plus_simulator").parent

## Load benchmark config

In [3]:

if APPROACH == "pretrained":
	config_path = Path(f"{PROJECT_ROOT_PATH}/config/pretrained/{BENCHMARK_NAME}.yaml")
elif APPROACH == "online":
	config_path = Path(f"{PROJECT_ROOT_PATH}/config/online/{BENCHMARK_NAME}.yaml")
	NUM_PRETRAINING_STEPS = 0
else:
	raise NotImplementedError(f"Approach {APPROACH} not implemented.")

with config_path.open("r") as f:
	CONFIG = yaml.safe_load(f)
	# pprint(CONFIG)


## Load dataset

In [4]:
input_df = read_files_from_folder(folder_path=f"{PROJECT_ROOT_PATH}/data/inference_outputs/{BENCHMARK_NAME}")
display(f"Loaded dataframe with {input_df.shape[0]} rows and {input_df.shape[1]} columns")
display(f"{len(input_df.columns.tolist())} available columns: {input_df.columns.tolist()}")
display(input_df.head())

'Loaded dataframe with 3270 rows and 14 columns'

"14 available columns: ['input_text', 'benchmark_name', 'label_small', 'acc_small', 'energy_consumption_small', 'inference_time_small', 'label_medium', 'acc_medium', 'energy_consumption_medium', 'inference_time_medium', 'label_large', 'acc_large', 'energy_consumption_large', 'inference_time_large']"

,input_text,benchmark_name,label_small,acc_small,energy_consumption_small,inference_time_small,label_medium,acc_medium,energy_consumption_medium,inference_time_medium,label_large,acc_large,energy_consumption_large,inference_time_large
doc_id,,,,,,,,,,,,,,
2500,does a bloody mary have alcohol in it,boolq,1.0,1.0,67.019,0.136539,1.0,1.0,135.085,0.400797,1.0,1.0,1051.882,2.096126
2501,is penn and teller fool us still on,boolq,1.0,1.0,30.183,0.131924,1.0,1.0,130.545,0.404249,1.0,1.0,1045.191,2.093104
2502,is a catbird the same as a mockingbird,boolq,1.0,1.0,64.417,0.128301,1.0,1.0,129.028,0.411191,1.0,1.0,1032.202,2.080621
2503,is the strip in the city of las vegas,boolq,0.0,0.0,24.690,0.138880,1.0,1.0,127.052,0.402043,1.0,1.0,1030.403,2.080694
2504,did the actor who played gandalf play dumbledore,boolq,1.0,1.0,34.529,0.107879,1.0,1.0,128.414,0.397378,1.0,1.0,1031.018,2.082012


## Load and configure classifier

In [6]:
classifier = MultilabelBERTClassifier(num_labels=3, **CONFIG["classifier_model"])
classifier.make_model_if_not_exists()
display(classifier.model.summary())

INFO:classifier.model:Using device: cpu
INFO:classifier.model:Initializing custom BERTClassifier: answerdotai/ModernBERT-base with 3 labels


RuntimeError: Failed to import transformers.models.modernbert.modeling_modernbert because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)